In [1]:
import sqlite3

In [2]:
conn = sqlite3.connect("factbook.db")
cur = conn.cursor()
cur.execute("PRAGMA table_info(facts)")
schema = cur.fetchall()
for row in schema:
    print(row)
conn.close()

(0, 'id', 'INTEGER', 1, None, 1)
(1, 'code', 'varchar(255)', 1, None, 0)
(2, 'name', 'varchar(255)', 1, None, 0)
(3, 'area', 'integer', 0, None, 0)
(4, 'area_land', 'integer', 0, None, 0)
(5, 'area_water', 'integer', 0, None, 0)
(6, 'population', 'integer', 0, None, 0)
(7, 'population_growth', 'float', 0, None, 0)
(8, 'birth_rate', 'float', 0, None, 0)
(9, 'death_rate', 'float', 0, None, 0)
(10, 'migration_rate', 'float', 0, None, 0)
(11, 'created_at', 'datetime', 0, None, 0)
(12, 'updated_at', 'datetime', 0, None, 0)


In [5]:
conn = sqlite3.connect('factbook.db')
query_plan = conn.cursor().execute("EXPLAIN QUERY PLAN SELECT * FROM facts").fetchall()
print(query_plan)
conn.close()

[(2, 0, 0, 'SCAN TABLE facts')]


In [10]:
conn = sqlite3.connect("factbook.db")
eqp = "EXPLAIN QUERY PLAN "
query_plan_one = conn.cursor().execute(eqp+"SELECT * FROM facts WHERE area > 40000;").fetchall()
query_plan_two = conn.execute(eqp + "SELECT area FROM facts WHERE area > 40000;").fetchall()
query_plan_three = conn.execute(eqp + "SELECT * FROM facts WHERE name = 'Czech Republic';").fetchall()
query_plan_four = conn.execute("EXPLAIN QUERY PLAN SELECT * FROM facts WHERE id = 15;").fetchall()
print(query_plan_one)
print(query_plan_two)
print(query_plan_three)
print(query_plan_four)
conn.close()

[(2, 0, 0, 'SCAN TABLE facts')]
[(2, 0, 0, 'SCAN TABLE facts')]
[(2, 0, 0, 'SCAN TABLE facts')]
[(2, 0, 0, 'SEARCH TABLE facts USING INTEGER PRIMARY KEY (rowid=?)')]


SQLite can use binary search to quickly find the corresponding row at that id value. Instead of performing a full table scan, SQLite would:

    use binary search to find the first row where the id value matches 15 in O(log N) time complexity and store this row in a temporary collection,
    advance to the next row to look for any more rows with the same id values and add those rows to the temporary collection,
    return the final collection of rows that matched.


In [9]:
conn = sqlite3.connect("factbook.db")
print(conn.cursor().execute("SELECT * FROM facts LIMIT 1;").fetchall())
conn.close()

[(1, 'af', 'Afghanistan', 652230, 652230, 0, 32564342, 2.32, 38.57, 13.89, 1.51, '2015-11-01 13:19:49.461734', '2015-11-01 13:19:49.461734')]


In [11]:
# CREATE INDEX IF NOT EXISTS area_indx ON facts(area);
conn = sqlite3.connect("factbook.db")
query_plan_six = conn.cursor().execute("explain query plan SELECT * FROM facts WHERE population > 10000;").fetchall()
print(query_plan_six)
conn.cursor().execute("CREATE INDEX IF NOT EXISTS pop_idx ON facts(population);")
query_plan_seven=conn.execute("explain query plan SELECT * FROM facts WHERE population > 10000;").fetchall()
print(query_plan_seven)
conn.close()

[(2, 0, 0, 'SCAN TABLE facts')]
[(3, 0, 0, 'SEARCH TABLE facts USING INDEX pop_idx (population>?)')]


In [12]:
conn = sqlite3.connect("factbook.db")
cur = conn.cursor()
query = "EXPLAIN QUERY PLAN SELECT * FROM facts WHERE population > 1000000 AND population_growth<0.05;"
query_plan_one = cur.execute(query).fetchall()
print(query_plan_one)
conn.close()

[(3, 0, 0, 'SEARCH TABLE facts USING INDEX pop_idx (population>?)')]


In [15]:
conn = sqlite3.connect('factbook.db')
cur = conn.cursor()
query = "create index if not exists pop_pop_growth_idx on facts(population, population_growth);"
cur.execute(query)
query_plan = cur.execute("explain query plan select * from facts where population > 1000000 and population_growth < 0.05;").fetchall()
print(query_plan)
conn.close()
                         

[(3, 0, 0, 'SEARCH TABLE facts USING INDEX pop_pop_growth_idx (population>?)')]
